In [1]:
import notebook_utils
from time import sleep
import json
from birddog.translate import (
    is_english, 
    translation,
    queue_translation,
    cancel_translation,
    is_translation_running)
from birddog.core import Archive

current working dir: /Users/jbrandt/code/birddog
Using AWS S3 bucket birddog-data for storage.


In [2]:
text = 'Перейменування: 1941-1945 Рейхскомісаріат України, м. Рівне.'

In [3]:
translation(text)

'Rename: 1941-1945 Reich Commissariat of Ukraine, Rivne.'

In [4]:
is_english(text)

False

In [5]:
is_english(translation(text))

True

In [6]:
translation([text, text, text, text])

['Rename: 1941-1945 Reich Commissariat of Ukraine, Rivne.',
 'Rename: 1941-1945 Reich Commissariat of Ukraine, Rivne.',
 'Rename: 1941-1945 Reich Commissariat of Ukraine, Rivne.',
 'Rename: 1941-1945 Reich Commissariat of Ukraine, Rivne.']

In [7]:
gen_batch = False
file_name = 'test/resources/dazho_d_descriptions.json'
if gen_batch:
    archive = Archive('DAZHO', 'D')
    translation_batch = [child[1]['text']['uk'] for child in archive.children]
    print('saving', len(translation_batch), 'items.')
    with open(file_name, 'w', encoding="utf8") as file:
        file.write(json.dumps(translation_batch))
else:
    with open(file_name, 'r', encoding="utf8") as file:
        translation_batch = json.loads(file.read())
        print('loading', len(translation_batch), 'items.')

loading 475 items.


In [8]:
def progress_cb(task_id, progress, total):
    print(f'progress[{task_id}]: {progress}/{total} {float(progress)/float(total)*100.:.1f}%')

In [9]:
def completion_cb(task_id, results):
    print(f'completed[{task_id}]:')
    for item in results[:10]:
        print('    ', item)

In [10]:
task_id = queue_translation(translation_batch[:80], progress_cb, completion_cb)
print('Queued', task_id)

Queued 7f4ff741-eeb0-4c29-9526-041e6483b35b


In [11]:
tasks = [
    queue_translation(translation_batch[batch[0]:batch[1]], progress_cb, completion_cb)
    for batch in ((0, 100), (100, 150), (150, 350), (350, 500)) ]
print('Queued:', tasks)

Queued: ['bb74cd92-c53a-44e5-bbd7-6a868a7510e6', 'af60dafc-dc1d-4f74-b63f-6b009ce20f3c', 'a6d5c985-26ff-47cf-80d6-1e1b3429d5da', 'ee6bd86f-2840-4645-b7ab-2f12fdc405df']


In [12]:
#sleep(10)
cancel_translation(tasks[3])

True

In [13]:
is_translation_running(tasks[3])

True

In [14]:
is_translation_running()

True

In [15]:
is_translation_running()

True

progress[a6d5c985-26ff-47cf-80d6-1e1b3429d5da]: 5/200 2.5%
progress[af60dafc-dc1d-4f74-b63f-6b009ce20f3c]: 5/50 10.0%
progress[ee6bd86f-2840-4645-b7ab-2f12fdc405df]: 5/125 4.0%
Task ee6bd86f-2840-4645-b7ab-2f12fdc405df cancelled.
progress[bb74cd92-c53a-44e5-bbd7-6a868a7510e6]: 5/100 5.0%
progress[7f4ff741-eeb0-4c29-9526-041e6483b35b]: 5/80 6.2%
progress[7f4ff741-eeb0-4c29-9526-041e6483b35b]: 10/80 12.5%
progress[bb74cd92-c53a-44e5-bbd7-6a868a7510e6]: 10/100 10.0%
progress[a6d5c985-26ff-47cf-80d6-1e1b3429d5da]: 10/200 5.0%
progress[af60dafc-dc1d-4f74-b63f-6b009ce20f3c]: 10/50 20.0%
progress[7f4ff741-eeb0-4c29-9526-041e6483b35b]: 15/80 18.8%
progress[bb74cd92-c53a-44e5-bbd7-6a868a7510e6]: 15/100 15.0%
progress[a6d5c985-26ff-47cf-80d6-1e1b3429d5da]: 15/200 7.5%
progress[af60dafc-dc1d-4f74-b63f-6b009ce20f3c]: 15/50 30.0%
progress[7f4ff741-eeb0-4c29-9526-041e6483b35b]: 20/80 25.0%
progress[af60dafc-dc1d-4f74-b63f-6b009ce20f3c]: 20/50 40.0%
progress[bb74cd92-c53a-44e5-bbd7-6a868a7510e6]: 20/